In [1]:
import sys
sys.path.append('../')

In [20]:
import json

import pandas
import numpy

import lib.utils as utils
import lib.timus_fetching as timus_fetching
import lib.difficulty_based_suggest as difficulty_based_suggest

In [3]:
timus = pandas.read_csv('../data/timus.csv', index_col=0)

/home/ec2-user/mount_point/tfnewenv/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
filtered_timus = utils.filter_timus_by_unique_accepted(timus, lambda cnt: cnt > 100)

In [11]:
def extend_problem_difficulties(timus,
                                problem_difficulties=dict(),
                                max_success_count=None
                               ):
    problem_difficulties = dict(problem_difficulties)
    all_problems = set(timus['problem_id'])
    success = 0
    for problem_id in all_problems:
        if problem_id in problem_difficulties:
            continue
        success += 1
        problem_difficulties[problem_id] = timus_fetching.fetch_problem_difficulty(problem_id)
        if max_success_count is not None and success >= max_success_count:
            break
    return problem_difficulties

In [ ]:
problem_difficulties = extend_problem_difficulties(timus)
with open('../data/problem_difficulties.json', 'w+') as f:
    f.write(json.dumps(problem_difficulties))

In [5]:
with open('../data/problem_difficulties.json') as f:
    problem_difficulties = json.loads(f.read())
problem_difficulties = {
    int(key): problem_difficulties[key]
    for key in problem_difficulties
}

In [ ]:
difficulty_based_scores = utils.get_algorithm_scores(
    timus=filtered_timus,
    suggest_for_timus=lambda t: difficulty_based_suggest.suggest(timus=t, top_count=10, problem_difficulties=problem_difficulties),
)

In [60]:
difficulty_based_scores

array([0.00134771, 0.00154959, 0.00122649, 0.00114242, 0.00099437,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.00028003, 0.00087057])

In [61]:
pandas.Series(difficulty_based_scores).describe()

count    12.000000
mean      0.000618
std       0.000623
min       0.000000
25%       0.000000
50%       0.000575
75%       0.001163
max       0.001550
dtype: float64